In [1]:
import numpy as np
import pandas as pd
import math
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

In [2]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import RMSprop

Using TensorFlow backend.


In [3]:
#save and load function
import pickle
def save_obj(obj, name ):
    with open('./'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('./' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [4]:
#with missing values
X_train_cnt = load_obj("data/X_train_LSTM_complete")
X_val_cnt =  load_obj("data/X_val_LSTM_complete")
X_test_cnt =  load_obj("data/X_test_LSTM_complete")

In [5]:
X_test_cnt.head()

,shop_id,item_id,item_category_id,top_categories_id,item_price_2,item_price_3,item_price_4,item_price_5,item_price_6,item_price_7,...,item_cnt_day_24,item_cnt_day_25,item_cnt_day_26,item_cnt_day_27,item_cnt_day_28,item_cnt_day_29,item_cnt_day_30,item_cnt_day_31,item_cnt_day_32,item_cnt_day_33
0,0.0,30.0,40.0,13.0,-0.135903,-0.135903,-0.135903,-0.135903,-0.135903,-0.135903,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2393,0.0,14798.0,40.0,13.0,-0.135903,-0.135903,-0.135903,-0.135903,-0.135903,-0.135903,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2394,0.0,14799.0,37.0,13.0,-0.135903,-0.135903,-0.135903,-0.135903,-0.135903,-0.135903,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2395,0.0,14800.0,40.0,13.0,-0.135903,-0.135903,-0.135903,-0.135903,-0.135903,-0.135903,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2396,0.0,14804.0,37.0,13.0,-0.135903,-0.135903,-0.135903,-0.135903,-0.135903,-0.135903,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
cols = X_train_cnt.columns
X_val_cnt.columns = cols

In [7]:
cols

Index(['shop_id', 'item_id', 'item_category_id', 'top_categories_id',
       'item_price_0', 'item_price_1', 'item_price_2', 'item_price_3',
       'item_price_4', 'item_price_5', 'item_price_6', 'item_price_7',
       'item_price_8', 'item_price_9', 'item_price_10', 'item_price_11',
       'item_price_12', 'item_price_13', 'item_price_14', 'item_price_15',
       'item_price_16', 'item_price_17', 'item_price_18', 'item_price_19',
       'item_price_20', 'item_price_21', 'item_price_22', 'item_price_23',
       'item_price_24', 'item_price_25', 'item_price_26', 'item_price_27',
       'item_price_28', 'item_price_29', 'item_price_30', 'item_price_31',
       'item_price_32', 'item_cnt_day_0', 'item_cnt_day_1', 'item_cnt_day_2',
       'item_cnt_day_3', 'item_cnt_day_4', 'item_cnt_day_5', 'item_cnt_day_6',
       'item_cnt_day_7', 'item_cnt_day_8', 'item_cnt_day_9', 'item_cnt_day_10',
       'item_cnt_day_11', 'item_cnt_day_12', 'item_cnt_day_13',
       'item_cnt_day_14', 'item_cnt_day

In [8]:
cols_test = cols.drop(['item_price_32','item_cnt_day_32'])

In [9]:
len(cols_test)

68

In [10]:
#prepare for 
X_test_cnt.columns = cols_test

In [11]:
cols_select = cols[37:]
cols_select

Index(['item_cnt_day_0', 'item_cnt_day_1', 'item_cnt_day_2', 'item_cnt_day_3',
       'item_cnt_day_4', 'item_cnt_day_5', 'item_cnt_day_6', 'item_cnt_day_7',
       'item_cnt_day_8', 'item_cnt_day_9', 'item_cnt_day_10',
       'item_cnt_day_11', 'item_cnt_day_12', 'item_cnt_day_13',
       'item_cnt_day_14', 'item_cnt_day_15', 'item_cnt_day_16',
       'item_cnt_day_17', 'item_cnt_day_18', 'item_cnt_day_19',
       'item_cnt_day_20', 'item_cnt_day_21', 'item_cnt_day_22',
       'item_cnt_day_23', 'item_cnt_day_24', 'item_cnt_day_25',
       'item_cnt_day_26', 'item_cnt_day_27', 'item_cnt_day_28',
       'item_cnt_day_29', 'item_cnt_day_30', 'item_cnt_day_31',
       'item_cnt_day_32'],
      dtype='object')

In [12]:
cols_select_test = cols_test[36:]
cols_select_test

Index(['item_cnt_day_0', 'item_cnt_day_1', 'item_cnt_day_2', 'item_cnt_day_3',
       'item_cnt_day_4', 'item_cnt_day_5', 'item_cnt_day_6', 'item_cnt_day_7',
       'item_cnt_day_8', 'item_cnt_day_9', 'item_cnt_day_10',
       'item_cnt_day_11', 'item_cnt_day_12', 'item_cnt_day_13',
       'item_cnt_day_14', 'item_cnt_day_15', 'item_cnt_day_16',
       'item_cnt_day_17', 'item_cnt_day_18', 'item_cnt_day_19',
       'item_cnt_day_20', 'item_cnt_day_21', 'item_cnt_day_22',
       'item_cnt_day_23', 'item_cnt_day_24', 'item_cnt_day_25',
       'item_cnt_day_26', 'item_cnt_day_27', 'item_cnt_day_28',
       'item_cnt_day_29', 'item_cnt_day_30', 'item_cnt_day_31'],
      dtype='object')

In [13]:
cols_select_price = cols[4:37]
cols_select_price

Index(['item_price_0', 'item_price_1', 'item_price_2', 'item_price_3',
       'item_price_4', 'item_price_5', 'item_price_6', 'item_price_7',
       'item_price_8', 'item_price_9', 'item_price_10', 'item_price_11',
       'item_price_12', 'item_price_13', 'item_price_14', 'item_price_15',
       'item_price_16', 'item_price_17', 'item_price_18', 'item_price_19',
       'item_price_20', 'item_price_21', 'item_price_22', 'item_price_23',
       'item_price_24', 'item_price_25', 'item_price_26', 'item_price_27',
       'item_price_28', 'item_price_29', 'item_price_30', 'item_price_31',
       'item_price_32'],
      dtype='object')

In [14]:
cols_select_price_test = cols[4:36]
cols_select_price_test

Index(['item_price_0', 'item_price_1', 'item_price_2', 'item_price_3',
       'item_price_4', 'item_price_5', 'item_price_6', 'item_price_7',
       'item_price_8', 'item_price_9', 'item_price_10', 'item_price_11',
       'item_price_12', 'item_price_13', 'item_price_14', 'item_price_15',
       'item_price_16', 'item_price_17', 'item_price_18', 'item_price_19',
       'item_price_20', 'item_price_21', 'item_price_22', 'item_price_23',
       'item_price_24', 'item_price_25', 'item_price_26', 'item_price_27',
       'item_price_28', 'item_price_29', 'item_price_30', 'item_price_31'],
      dtype='object')

In [15]:
cols_select_shop = cols[0]
cols_select_shop

'shop_id'

In [16]:
cols_select_item = cols[1]
cols_select_item

'item_id'

In [17]:
cols_select_cat1 = cols[2]
cols_select_cat1

'item_category_id'

In [18]:
cols_select_cat2 = cols[3]
cols_select_cat2

'top_categories_id'

In [19]:
cols_dummy = list(range(33))
len(cols_dummy)

33

In [20]:
cols_test_dummy = cols_dummy[:-1]
len(cols_test_dummy)

32

In [175]:
X_train_cnt_partial = X_train_cnt[cols_select]
X_val_cnt_partial = X_val_cnt[cols_select]
X_test_cnt_partial = X_test_cnt[cols_select_test]

In [176]:
X_train_cnt_partial.columns = cols_dummy
X_val_cnt_partial.columns = cols_dummy
X_test_cnt_partial.columns = cols_test_dummy

In [177]:
X_train_cnt_price = X_train_cnt[cols_select_price]
X_val_cnt_price = X_val_cnt[cols_select_price]
X_test_cnt_price = X_test_cnt[cols_select_price_test]

In [178]:
X_train_cnt_price.columns = cols_dummy
X_val_cnt_price.columns = cols_dummy
X_test_cnt_price.columns = cols_test_dummy

In [179]:
X_train_cnt_price.head()

,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
0,-0.138812,0.378800,-0.138812,-0.138812,-0.138812,-0.138812,-0.138812,-0.138812,-0.138812,-0.138812,...,-0.138812,-0.138812,-0.138812,-0.138812,-0.138812,-0.138812,-0.138812,-0.138812,-0.138812,-0.138812
2393,0.384660,0.384660,-0.138812,-0.138812,-0.138812,-0.138812,-0.138812,-0.138812,-0.138812,-0.138812,...,-0.138812,-0.138812,-0.138812,-0.138812,-0.138812,-0.138812,-0.138812,-0.138812,-0.138812,-0.138812
2394,1.144475,1.144475,-0.138812,-0.138812,-0.138812,-0.138812,-0.138812,-0.138812,-0.138812,-0.138812,...,-0.138812,-0.138812,-0.138812,-0.138812,-0.138812,-0.138812,-0.138812,-0.138812,-0.138812,-0.138812
2395,0.009635,-0.138812,-0.138812,-0.138812,-0.138812,-0.138812,-0.138812,-0.138812,-0.138812,-0.138812,...,-0.138812,-0.138812,-0.138812,-0.138812,-0.138812,-0.138812,-0.138812,-0.138812,-0.138812,-0.138812
2396,1.072205,1.072205,-0.138812,-0.138812,-0.138812,-0.138812,-0.138812,-0.138812,-0.138812,-0.138812,...,-0.138812,-0.138812,-0.138812,-0.138812,-0.138812,-0.138812,-0.138812,-0.138812,-0.138812,-0.138812


In [180]:
X_val_cnt_price.head()

,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
0,0.374659,-0.13664,-0.13664,-0.13664,-0.13664,-0.13664,-0.13664,-0.13664,-0.13664,-0.13664,...,-0.13664,-0.13664,-0.13664,-0.13664,-0.13664,-0.13664,-0.13664,-0.13664,-0.13664,-0.13664
2393,0.380447,-0.13664,-0.13664,-0.13664,-0.13664,-0.13664,-0.13664,-0.13664,-0.13664,-0.13664,...,-0.13664,-0.13664,-0.13664,-0.13664,-0.13664,-0.13664,-0.13664,-0.13664,-0.13664,-0.13664
2394,1.130996,-0.13664,-0.13664,-0.13664,-0.13664,-0.13664,-0.13664,-0.13664,-0.13664,-0.13664,...,-0.13664,-0.13664,-0.13664,-0.13664,-0.13664,-0.13664,-0.13664,-0.13664,-0.13664,-0.13664
2395,-0.136640,-0.13664,-0.13664,-0.13664,-0.13664,-0.13664,-0.13664,-0.13664,-0.13664,-0.13664,...,-0.13664,-0.13664,-0.13664,-0.13664,-0.13664,-0.13664,-0.13664,-0.13664,-0.13664,-0.13664
2396,1.059607,-0.13664,-0.13664,-0.13664,-0.13664,-0.13664,-0.13664,-0.13664,-0.13664,-0.13664,...,-0.13664,-0.13664,-0.13664,-0.13664,-0.13664,-0.13664,-0.13664,-0.13664,-0.13664,-0.13664


In [181]:
X_train_cnt_shop = X_train_cnt[cols_select_shop]
X_val_cnt_shop = X_val_cnt[cols_select_shop]
X_test_cnt_shop = X_test_cnt[cols_select_shop]

X_train_cnt_item = X_train_cnt[cols_select_item]
X_val_cnt_item = X_val_cnt[cols_select_item]
X_test_cnt_item = X_test_cnt[cols_select_item]


##### Create Dummy Variables for static values such as shop id and item id for each time sequence

In [182]:
X_train_cnt_shop_df = pd.DataFrame(X_train_cnt_shop.copy())
for c in cols_dummy:
    X_train_cnt_shop_df[c] =  X_train_cnt_shop
X_train_cnt_shop_df= X_train_cnt_shop_df.drop(columns = ["shop_id"]) 

In [183]:
X_val_cnt_shop_df = pd.DataFrame(X_val_cnt_shop.copy())
for c in cols_dummy:
    X_val_cnt_shop_df[c] =  X_val_cnt_shop
X_val_cnt_shop_df= X_val_cnt_shop_df.drop(columns = ["shop_id"]) 

In [184]:
X_test_cnt_shop_df = pd.DataFrame(X_test_cnt_shop.copy())
for c in cols_test_dummy:
    X_test_cnt_shop_df[c] =  X_test_cnt_shop
X_test_cnt_shop_df= X_test_cnt_shop_df.drop(columns = ["shop_id"]) 

In [185]:
X_train_cnt_item_df = pd.DataFrame(X_train_cnt_item.copy())
for c in cols_dummy:
    X_train_cnt_item_df[c] =  X_train_cnt_item
X_train_cnt_item_df= X_train_cnt_item_df.drop(columns = ["item_id"]) 

In [186]:
X_val_cnt_item_df = pd.DataFrame(X_val_cnt_item.copy())
for c in cols_dummy:
    X_val_cnt_item_df[c] =  X_val_cnt_item
X_val_cnt_item_df= X_val_cnt_item_df.drop(columns = ["item_id"]) 

In [187]:
X_test_cnt_item_df = pd.DataFrame(X_test_cnt_item.copy())
for c in cols_test_dummy:
    X_test_cnt_item_df[c] =  X_test_cnt_item
X_test_cnt_item_df= X_test_cnt_item_df.drop(columns = ["item_id"]) 

##### creating summy variables for the 2 category labels

In [188]:
X_train_cnt_cat1 = X_train_cnt[cols_select_cat1]
X_val_cnt_cat1 = X_val_cnt[cols_select_cat1]
X_test_cnt_cat1 = X_test_cnt[cols_select_cat1]

X_train_cnt_cat2 = X_train_cnt[cols_select_cat2]
X_val_cnt_cat2 = X_val_cnt[cols_select_cat2]
X_test_cnt_cat2 = X_test_cnt[cols_select_cat2]


In [189]:
X_train_cnt_category0 = pd.DataFrame(X_train_cnt_cat1.copy())
for c in cols_dummy:
    X_train_cnt_category0[c] =  X_train_cnt_cat1
X_train_cnt_category0 = X_train_cnt_category0.drop(columns = ["item_category_id"])   

In [190]:
X_train_cnt_category0.head()

,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
0,40.0,40.0,40.0,40.0,40.0,40.0,40.0,40.0,40.0,40.0,...,40.0,40.0,40.0,40.0,40.0,40.0,40.0,40.0,40.0,40.0
2393,40.0,40.0,40.0,40.0,40.0,40.0,40.0,40.0,40.0,40.0,...,40.0,40.0,40.0,40.0,40.0,40.0,40.0,40.0,40.0,40.0
2394,37.0,37.0,37.0,37.0,37.0,37.0,37.0,37.0,37.0,37.0,...,37.0,37.0,37.0,37.0,37.0,37.0,37.0,37.0,37.0,37.0
2395,40.0,40.0,40.0,40.0,40.0,40.0,40.0,40.0,40.0,40.0,...,40.0,40.0,40.0,40.0,40.0,40.0,40.0,40.0,40.0,40.0
2396,37.0,37.0,37.0,37.0,37.0,37.0,37.0,37.0,37.0,37.0,...,37.0,37.0,37.0,37.0,37.0,37.0,37.0,37.0,37.0,37.0


In [191]:
X_val_cnt_category0 = pd.DataFrame(X_val_cnt_cat1.copy())
for c in cols_dummy:
    X_val_cnt_category0[c] =  X_val_cnt_cat1
X_val_cnt_category0 = X_val_cnt_category0.drop(columns = ["item_category_id"]) 

In [192]:
X_val_cnt_category0.head()

,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
0,40.0,40.0,40.0,40.0,40.0,40.0,40.0,40.0,40.0,40.0,...,40.0,40.0,40.0,40.0,40.0,40.0,40.0,40.0,40.0,40.0
2393,40.0,40.0,40.0,40.0,40.0,40.0,40.0,40.0,40.0,40.0,...,40.0,40.0,40.0,40.0,40.0,40.0,40.0,40.0,40.0,40.0
2394,37.0,37.0,37.0,37.0,37.0,37.0,37.0,37.0,37.0,37.0,...,37.0,37.0,37.0,37.0,37.0,37.0,37.0,37.0,37.0,37.0
2395,40.0,40.0,40.0,40.0,40.0,40.0,40.0,40.0,40.0,40.0,...,40.0,40.0,40.0,40.0,40.0,40.0,40.0,40.0,40.0,40.0
2396,37.0,37.0,37.0,37.0,37.0,37.0,37.0,37.0,37.0,37.0,...,37.0,37.0,37.0,37.0,37.0,37.0,37.0,37.0,37.0,37.0


In [193]:
X_test_cnt_category0 = pd.DataFrame(X_test_cnt_cat1.copy())
for c in cols_test_dummy:
    X_test_cnt_category0[c] =  X_test_cnt_cat1
X_test_cnt_category0 = X_test_cnt_category0.drop(columns = ["item_category_id"]) 

In [194]:
X_train_cnt_category1 = pd.DataFrame(X_train_cnt_cat2.copy())
for c in cols_dummy:
    X_train_cnt_category1[c] =  X_train_cnt_cat2
X_train_cnt_category1 = X_train_cnt_category1.drop(columns = ["top_categories_id"])   

X_val_cnt_category1 = pd.DataFrame(X_val_cnt_cat2.copy())
for c in cols_dummy:
    X_val_cnt_category1[c] =  X_val_cnt_cat2
X_val_cnt_category1 = X_val_cnt_category1.drop(columns = ["top_categories_id"])   

In [195]:
X_test_cnt_category1 = pd.DataFrame(X_test_cnt_cat2.copy())
for c in cols_test_dummy:
    X_test_cnt_category1[c] =  X_test_cnt_cat2
X_test_cnt_category1 = X_test_cnt_category1.drop(columns = ["top_categories_id"])   

In [196]:
import itertools

In [197]:
#create the month feature for train data
def create_seasonal_feature(total_month_num, start_month):
    month_feature = []
    n_year = int(total_month_num//12)
    m_residual = int(total_month_num % 12)
    if n_year > 0:
        for i in range(n_year):
            month_feature.append(list(range(start_month,start_month+12)))
        if m_residual > 0:
            month_feature.append(list(range(start_month,m_residual+start_month)))
    else:
        #assume in this case there's at least one month of data
        month_feature.append(list(range(start_month,m_residual+start_month))) 
        
    month_feature_final = list(itertools.chain(*month_feature))
    
    return month_feature_final

In [198]:
X_train_cnt_partial = pd.concat([X_train_cnt_partial,X_train_cnt_price])

In [199]:
X_train_cnt_partial = pd.concat([X_train_cnt_partial,X_train_cnt_shop_df])
#X_train_cnt_partial = pd.concat([X_train_cnt_partial,X_train_cnt_item_df])
#X_train_cnt_partial = pd.concat([X_train_cnt_partial,X_train_cnt_category0])
X_train_cnt_partial = pd.concat([X_train_cnt_partial,X_train_cnt_category1])

In [200]:
#create the list for data
all_months = create_seasonal_feature(len(cols_dummy)+2, 1)
#train_months_df = pd.DataFrame(train_months.T,columns=cols_dummy)

In [201]:
train_months = all_months[:len(cols_dummy)]

In [202]:
#add month to the last row
X_train_cnt_partial.iloc[-1] = train_months

In [203]:
X_val_cnt_partial = pd.concat([X_val_cnt_partial,X_val_cnt_price])

In [204]:
X_val_cnt_partial = pd.concat([X_val_cnt_partial,X_val_cnt_shop_df])
#X_val_cnt_partial  = pd.concat([X_val_cnt_partial,X_val_cnt_item_df])
#X_val_cnt_partial  = pd.concat([X_val_cnt_partial,X_val_cnt_category0])
X_val_cnt_partial  = pd.concat([X_val_cnt_partial,X_val_cnt_category1])

In [205]:
#create the list for data
val_months = all_months[1:len(cols_dummy)+1]
X_val_cnt_partial.iloc[-1] = val_months

In [52]:
#X_test has no target value, it's only for prediction

In [206]:
X_test_cnt_partial = pd.concat([X_test_cnt_partial,X_test_cnt_price])

In [207]:
X_test_cnt_partial  = pd.concat([X_test_cnt_partial,X_test_cnt_shop_df])
#X_test_cnt_partial  = pd.concat([X_test_cnt_partial,X_test_cnt_item_df])
#X_test_cnt_partial  = pd.concat([X_test_cnt_partial,X_test_cnt_category0])
X_test_cnt_partial  = pd.concat([X_test_cnt_partial,X_test_cnt_category1])

In [208]:
#create the list for test data
test_months = all_months[2:-1]
X_test_cnt_partial.iloc[-1] = test_months

In [209]:
X_train = X_train_cnt_partial.iloc[:,:-1]
y_train = X_train_cnt_partial.iloc[:,-1]

In [210]:
X_train.head()

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,0.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2393,10.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2394,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2395,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2396,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [211]:
X_val = X_val_cnt_partial.iloc[:,:-1]
y_val = X_val_cnt_partial.iloc[:,-1]

In [212]:
X_test = X_test_cnt_partial.copy()

In [213]:
X_train_array = np.array(X_train)
y_train_array = np.array(y_train)

In [214]:
X_val_array = np.array(X_val)
y_val_array = np.array(y_val)

In [215]:
X_test_array =  np.array(X_test)

In [216]:
X_train_array.shape

(2107680, 32)

In [217]:
y_train_array.shape

(2107680,)

In [218]:
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back+1):
        a = dataset[i:(i+look_back), :]
        dataX.append(a)
        dataY.append(dataset[i + look_back - 1, :])
    return np.array(dataX), np.array(dataY)

In [219]:
X_trial,y_trial = create_dataset(X_train_array.T,look_back = 4)

In [220]:
X_trial.shape #27 time sequence samples, each sample has 5 time steps, every sample has 424124 observations

(29, 4, 2107680)

In [221]:
y_trial.shape

(29, 2107680)

In [222]:
n_observation = len(X_train_cnt)
n_observation

526920

In [70]:
#save the train data
#save_obj(X_trial,"data/X_trial")
#save_obj(y_trial,"data/y_trial")

In [71]:
#X_trial = load_obj("data/X_trial")
#y_trial = load_obj("data/y_trial")

In [223]:
n_model = X_trial.shape[2]
n_model

2107680

In [224]:
model = Sequential()
#model.add(LSTM(150, input_shape=(5, n_observation),return_sequences=True)) #5 timesteps, #num of observations
model.add(LSTM(50, input_shape=(4, n_model), return_sequences=True))
model.add(Dropout(0.2)) #control for overfitting
model.add(LSTM(32, return_sequences=False))
model.add(Dense(n_model, activation='linear'))

optimizer = "adam"
#optimizer = RMSprop(lr=0.005)
model.compile(loss='mean_squared_error', optimizer=optimizer)

In [225]:
X_trial.shape

(29, 4, 2107680)

In [226]:
X_train_cnt.isnull().any()[2]

False

In [227]:
model.fit(X_trial, y_trial, batch_size=10, epochs=100)

Epoch 1/100
29/29 [==============================] - 15s 509ms/step - loss: 366.3746
Epoch 2/100
29/29 [==============================] - 14s 484ms/step - loss: 365.9956
Epoch 3/100
29/29 [==============================] - 14s 487ms/step - loss: 365.4642
Epoch 4/100
29/29 [==============================] - 14s 485ms/step - loss: 364.7797
Epoch 5/100
29/29 [==============================] - 14s 482ms/step - loss: 363.8681
Epoch 6/100
29/29 [==============================] - 14s 490ms/step - loss: 362.7080
Epoch 7/100
29/29 [==============================] - 14s 487ms/step - loss: 361.3555
Epoch 8/100
29/29 [==============================] - 14s 488ms/step - loss: 359.6739
Epoch 9/100
29/29 [==============================] - 14s 485ms/step - loss: 357.7202
Epoch 10/100
29/29 [==============================] - 14s 486ms/step - loss: 355.4860
Epoch 11/100
29/29 [==============================] - 14s 490ms/step - loss: 353.0738
Epoch 12/100
29/29 [==============================] - 14s 484ms

In [228]:
X_val_reshape,y_val_reshape = create_dataset(X_val_array.T,look_back =4)

In [229]:
X_val_reshape.shape

(29, 4, 2107680)

In [230]:
y_val_reshape.shape

(29, 2107680)

In [313]:
#save the validation data
#save_obj(X_val_reshape,"data/X_val_reshape")
#save_obj(y_val_reshape,"data/y_val_reshape")

In [231]:
y_val_pred = model.predict(X_val_reshape)

In [232]:
y_val_pred.shape

(29, 2107680)

In [76]:
#complete dataset, price+cnt+seasonal factor, 12 month window
rmse_LSTM = np.sqrt(mean_squared_error(y_val_reshape[-1][:int(n_observation)],y_val_pred[-1][:int(n_observation)]))
rmse_LSTM

4.387210470356987

In [77]:
val_pred_clip = y_val_pred[-1][:int(n_observation)].clip(0,20)
y_val_array_clip = y_val_reshape[-1][:int(n_observation)].clip(0,20)
rmse_LSTM_clip = np.sqrt(mean_squared_error(y_val_array_clip,val_pred_clip))
rmse_LSTM_clip

0.7549364791275444

In [115]:
#save test results
X_test_reshape,y_test = create_dataset(X_test_array.T,look_back = 4)
y_test_pred = model.predict(X_test_reshape)
save_obj(y_test_pred[-1][:int(n_observation)], "data/y_test_pred_cnt_complete_price_cnt_12")

In [172]:
#complete data all factors, 3 months window
rmse_LSTM = np.sqrt(mean_squared_error(y_val_reshape[-1][:int(n_observation)],y_val_pred[-1][:int(n_observation)]))
rmse_LSTM

4.3664611338665305

In [173]:
val_pred_clip = y_val_pred[-1][:int(n_observation)].clip(0,20)
y_val_array_clip = y_val_reshape[-1][:int(n_observation)].clip(0,20)
rmse_LSTM_clip = np.sqrt(mean_squared_error(y_val_array_clip,val_pred_clip))
rmse_LSTM_clip

0.8129897558491831

In [174]:
X_test_reshape,y_test = create_dataset(X_test_array.T,look_back = 3)
y_test_pred = model.predict(X_test_reshape)
save_obj(y_test_pred[-1][:int(n_observation)], "data/y_test_pred_complete_everything_3month")

In [233]:
#complete data, cnt+price+seasonal effect+shop+top_catergories, 4 months
rmse_LSTM = np.sqrt(mean_squared_error(y_val_reshape[-1][:int(n_observation)],y_val_pred[-1][:int(n_observation)]))
rmse_LSTM

4.391848550465688

In [234]:
val_pred_clip = y_val_pred[-1][:int(n_observation)].clip(0,20)
y_val_array_clip = y_val_reshape[-1][:int(n_observation)].clip(0,20)
rmse_LSTM_clip = np.sqrt(mean_squared_error(y_val_array_clip,val_pred_clip))
rmse_LSTM_clip

0.7815176474328304

In [235]:
X_test_reshape,y_test = create_dataset(X_test_array.T,look_back = 4)
y_test_pred = model.predict(X_test_reshape)
save_obj(y_test_pred[-1][:int(n_observation)], "data/y_test_pred_complete_cnt_price_seasonal_shop_cat1_4month")

In [224]:
#with only cnt,price and month (seasonal effect), 12 months window
rmse_LSTM = np.sqrt(mean_squared_error(y_val_reshape[-1][:int(n_observation)],y_val_pred[-1][:int(n_observation)]))
rmse_LSTM

4.896569445285344

In [225]:
val_pred_clip = y_val_pred[-1][:int(n_observation)].clip(0,20)
y_val_array_clip = y_val_reshape[-1][:int(n_observation)].clip(0,20)
rmse_LSTM_clip = np.sqrt(mean_squared_error(y_val_array_clip,val_pred_clip))
rmse_LSTM_clip

0.8293623938192951

In [113]:
#complete dataset,with only cnt,price and month (seasonal effect),shop,top category, 4 months window
rmse_LSTM = np.sqrt(mean_squared_error(y_val_reshape[-1][:int(n_observation)],y_val_pred[-1][:int(n_observation)]))
rmse_LSTM

4.379205522900886

In [114]:
val_pred_clip = y_val_pred[-1][:int(n_observation)].clip(0,20)
y_val_array_clip = y_val_reshape[-1][:int(n_observation)].clip(0,20)
rmse_LSTM_clip = np.sqrt(mean_squared_error(y_val_array_clip,val_pred_clip))
rmse_LSTM_clip

0.7980059393925408

In [ ]:
#save test results
X_test_reshape,y_test = create_dataset(X_test_array.T,look_back = 4)
y_test_pred = model.predict(X_test_reshape)
save_obj(y_test_pred[-1][:int(n_observation)], "data/y_test_pred_cnt_complete_price_cnt_shop_topcat_4")